In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
df = pd.read_csv("data.csv")    
df.head()

,Crash ID,Record Type,Vehicle ID,Participant ID,Participant Display Seq#,Vehicle Coded Seq#,Participant Vehicle Seq#,Serial #,Crash Month,Crash Day,...,Participant Cause 2 Code,Participant Cause 3 Code,Participant Event 1 Code,Participant Event 2 Code,Participant Event 3 Code,BAC Test Results Code,Alcohol Use Reported,Drug Use Reported,Participant Marijuana Use Reported,Participant Striker Flag
0,1809119,1,NaN,NaN,NaN,NaN,NaN,99992.0,3.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1809119,2,3409578.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1809119,3,3409578.0,3887884.0,1.0,1.0,1.0,NaN,NaN,NaN,...,NaN,0.0,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1809119,2,3409579.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1809119,3,3409579.0,3887885.0,2.0,2.0,1.0,NaN,NaN,NaN,...,NaN,NaN,9.0,9.0,0.0,NaN,NaN,NaN,NaN,NaN


In [3]:
crashes_df = df[df['Record Type'] == 1]
vehicles_df = df[df['Record Type'] == 2]
participants_df = df[df['Record Type'] == 3]

crashes_df_len = len(crashes_df)
vehicles_df_len = len(vehicles_df)
participants_df_len = len(participants_df)

In [4]:
check_all_rows = lambda column_name: lambda df: len(df[df[column_name].isnull()])
types_df = [crashes_df, vehicles_df, participants_df]
lens_df = [crashes_df_len, vehicles_df_len, participants_df_len]

Validate Every record has a crash id

In [5]:
num_of_nan_crashid = check_all_rows('Crash ID')
crashid_not_nan = [num_of_nan_crashid(df) for df in types_df]

print("# of nan serial number", crashid_not_nan)
print("# of entries", lens_df)

# of nan serial number [0, 0, 0]
# of entries [508, 1015, 1216]


Validate Every record has a serial number, the running result show that only every type 1 record has a serial number

In [6]:
num_of_nan_serial = check_all_rows('Serial #')
serial_not_nan = [num_of_nan_serial(df) for df in types_df]

print("# of nan serial number", serial_not_nan)
print("# of entries", lens_df)

# of nan serial number [0, 1015, 1216]
# of entries [508, 1015, 1216]


Validate Every serial number is between 0 and 99999

In [7]:
serial_in_range = crashes_df[(crashes_df['Serial #'] > 0) & (crashes_df['Serial #'] < 99999)]

print("# of serial in [0,99999]", len(serial_in_range))
print("# of entries", crashes_df_len)

# of serial in [0,99999] 508
# of entries 508


Validate Every entered posted speed is between 0 and 65. First find which table contains posted speed: only type 1 table (crashes df) contains posted speed; then check all the entries in crashes df

In [8]:
num_of_nan_posted_speed = check_all_rows('Posted Speed Limit')
posted_speed_not_nan = [num_of_nan_posted_speed(df) for df in types_df]

print(posted_speed_not_nan)
print("# of entries", lens_df)

[0, 1015, 1216]
# of entries [508, 1015, 1216]


In [9]:
posted_speed_in_range = crashes_df[(crashes_df['Posted Speed Limit'] >= 0) & (crashes_df['Posted Speed Limit'] <= 65)]

print("# of posted speed in [0, 65]", len(posted_speed_in_range))
print("# of entries", crashes_df_len)

# of posted speed in [0, 65] 508
# of entries 508


Validate When Highway Number is entered, Impact Location Code must be a Numeric value <= 14. The running result shows that there are 3 records don't have a numeric value.

In [10]:
num_of_nan_highway_num = check_all_rows('Highway Number')
num_of_nan_impact_loc = check_all_rows('Impact Location')

highway_num_not_nan = [num_of_nan_highway_num(df) for df in types_df]
impact_loc_not_nan = [num_of_nan_impact_loc(df) for df in types_df]

print("# of nan highway number", highway_num_not_nan)
print("# of nan impact location", impact_loc_not_nan)
print("# of entries", lens_df)

# of nan highway number [0, 1015, 1216]
# of nan impact location [0, 1015, 1216]
# of entries [508, 1015, 1216]


In [11]:
check_numeric = crashes_df['Impact Location'].str.isnumeric()
check_numeric.loc[check_numeric == False]

192     False
1318    False
1879    False
Name: Impact Location, dtype: bool

Validate When Crash Type Code = 4 (Train), one of Crash-level Event code values must be 111, 112, 113, 015 or 016. The running result shows that only 7 records satisify this assertion out of 417 records with Crash Type Code = 4 

In [12]:
crash_type_code_4 = crashes_df[crashes_df['Crash Type'] == 4]
options = [111, 112, 113, 15, 16]
crash_lvl_evt_satisified = crash_type_code_4[((crash_type_code_4['Crash Level Event 1 Code'].isin(options))
                                            | (crash_type_code_4['Crash Level Event 2 Code'].isin(options))
                                            | (crash_type_code_4['Crash Level Event 3 Code'].isin(options)))]

print("# of satisified records", len(crash_lvl_evt_satisified))
print("# of entries", len(crash_type_code_4))

# of satisified records 7
# of entries 417


Validate When crash type is 1 or 2, at least two records with distinct vehicle ID must associate with that crash ID. The running result show that crash ID 1809229, 1809637, 1810874 and 1816804 only associate with one vehicle.

In [22]:
crash_type_code_1_or_2 = crashes_df[crashes_df['Crash Type'].isin([1, 2])]
print(crash_type_code_1_or_2['Crash ID'])
vehicles = vehicles_df[vehicles_df['Crash ID'].isin(crash_type_code_1_or_2['Crash ID'])]
vehicles.groupby(['Crash ID']).agg(['count'])[['Vehicle ID']]

0     1809119
5     1809229
9     1809637
12    1810874
16    1812266
26    1815964
31    1816804
Name: Crash ID, dtype: int64


,Vehicle ID
,count
Crash ID,
1809119,2
1809229,1
1809637,1
1810874,1
1812266,4
1815964,2
1816804,1


Validate For each crash ID, there must be at least three records associated with it (record type 1, record type 2, and record type 3). The following code check each row of joined 3 table if each row the crash id is identical after remove duplicates. The resulting "eq" operator returns a dataframe that shows that all the rows are "True" (count 508, unique 1 with value True").

In [50]:
crashes_df_ids = crashes_df[['Crash ID']].reset_index()
vehicles_df_ids = vehicles_df[['Crash ID']].drop_duplicates().reset_index()
participants_df_ids = participants_df[['Crash ID']].drop_duplicates().reset_index()

joined = crashes_df_ids.join(vehicles_df_ids, rsuffix='_vehicles').join(participants_df_ids, rsuffix='_participants')

joined[['Crash ID', 'Crash ID_vehicles', 'Crash ID_participants']].eq(joined.iloc[:, 1], axis=0).describe()

,Crash ID,Crash ID_vehicles,Crash ID_participants
count,508,508,508
unique,1,1,1
top,True,True,True
freq,508,508,508


Validate Every crash has at least one associated vehicle ID.

In [69]:
distinct_vehicle_counter = vehicles_df.groupby(['Crash ID']).agg(['count'])[['Vehicle ID']]
distinct_vehicle_counter.describe().loc['min']

Vehicle ID  count    1.0
Name: min, dtype: float64

Validate Every crash has a unique combination of country code, year with serial number

In [78]:
check_unique_comb = (
    crashes_df[['County Code', 'Serial #', 'Crash Year']]
        .groupby(['County Code', 'Serial #', 'Crash Year'])
        .size()
        .reset_index()
        .rename(columns={0:'count'})
)
check_unique_comb[['count']].eq(1).describe()

,count
count,508
unique,1
top,True
freq,508
